In [1]:
# Instalar pip install scikit-learn
# IMPORTAR LIBRERÍAS

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [2]:

# CARGAR DATASETS

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [3]:
# SELECCIÓN DE VARIABLES

features = [
    'FAMI_EDUCACIONMADRE', 'ESTU_VALORMATRICULAUNIVERSIDAD', 'FAMI_ESTRATOVIVIENDA',
    'FAMI_TIENEINTERNET', 'FAMI_TIENECOMPUTADOR', 'FAMI_EDUCACIONPADRE',
    'ESTU_HORASSEMANATRABAJA', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL',
    'ESTU_PAGOMATRICULAPROPIO', 'coef_1', 'coef_2', 'coef_3', 'coef_4'
]
target = 'RENDIMIENTO_GLOBAL'


# MAPEO MATRÍCULA

matricula_map = {
    'Entre 1 millón y menos de 2.5 millones': 1.75,
    'Entre 2.5 millones y menos de 4 millones': 3.25,
    'Menos de 500 mil': 0.250,
    'Entre 500 mil y menos de 1 millón': 0.75,
    'Entre 4 millones y menos de 5.5 millones': 4.75,
    'Más de 7 millones': 7.75,
    'Entre 5.5 millones y menos de 7 millones': 6.25,
    'No pagó matrícula': 0,
    'no info': -1
}

In [4]:
# PREPROCESAMIENTO TRAIN

df = train_df[features + [target]].copy().fillna('no info')
df['ESTU_VALORMATRICULAUNIVERSIDAD'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].map(matricula_map)
df[target] = df[target].map({'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3})
df = pd.get_dummies(df)


# PREPROCESAMIENTO TEST

test_ids = test_df['ID']
test_df_proc = test_df[features].copy().fillna('no info')
test_df_proc['ESTU_VALORMATRICULAUNIVERSIDAD'] = test_df_proc['ESTU_VALORMATRICULAUNIVERSIDAD'].map(matricula_map)
test_df_proc = pd.get_dummies(test_df_proc)

# Alinear columnas de test con las de train
X_train = df.drop(columns=[target])
y_train = df[target]
X_test = test_df_proc.reindex(columns=X_train.columns, fill_value=0)


In [5]:

# MODELO RANDOM FOREST

rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=15,
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)


# VALIDACIÓN

scores = cross_val_score(rf_model, X_train, y_train, cv=5, scoring='accuracy')
print(f"✅ Accuracy promedio con Random Forest: {scores.mean():.4f}")


# PREDICCIÓN Y SUBMISSION

y_pred = rf_model.predict(X_test)

inv_map = {0: 'bajo', 1: 'medio-bajo', 2: 'medio-alto', 3: 'alto'}
y_pred_labels = [inv_map[i] for i in y_pred]

submission = pd.DataFrame({
    'ID': test_ids,
    'RENDIMIENTO_GLOBAL': y_pred_labels
})

submission.to_csv('submission.csv', index=False)
print("✅ Archivo 'submission.csv' generado exitosamente.")


✅ Accuracy promedio con Random Forest: 0.3895
✅ Archivo 'submission.csv' generado exitosamente.
